## Importing the Dependencies

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import os
os.listdir("data")


## LOAD DATA

In [ ]:
import pandas as pd

df = pd.read_csv('data/benin-malanville.csv', parse_dates=['Timestamp'])
df.head()

## Summary Statistics & Missing-Value Report

In [ ]:
df.describe()

missing = df.isna().sum()
print(missing[missing > 0])


NameError: name 'df' is not defined

## Outlier Detection & Basic Cleaning

In [ ]:
from scipy import stats
import numpy as np

cols_to_check = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Compute Z-scores
z_scores = np.abs(stats.zscore(df[cols_to_check], nan_policy='omit'))
outlier_rows = (z_scores > 3).any(axis=1)

# Flag rows with outliers
df['Outlier'] = outlier_rows

# Fill missing values in key columns with median
df[cols_to_check] = df[cols_to_check].fillna(df[cols_to_check].median())

## Export Cleaned Data

In [ ]:
df.to_csv('data/benin-malanville.csv', index=False)

## Time Series Analysis

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(df['Timestamp'], df['GHI'], label='GHI')
plt.plot(df['Timestamp'], df['DNI'], label='DNI')
plt.plot(df['Timestamp'], df['DHI'], label='DHI')
plt.xlabel('Timestamp')
plt.ylabel('Solar Irradiance')
plt.title('Solar Irradiance Over Time')
plt.legend()
plt.show()


## Cleaning Impact

In [ ]:
df.groupby('Outlier')[['ModA','ModB']].mean().plot(kind='bar')
plt.title('Average ModA & ModB Pre/Post-Cleaning')
plt.show()


## Correlation & Relationship Analysis

In [ ]:
import seaborn as sns

# Heatmap
sns.heatmap(df[['GHI','DNI','DHI','ModA','ModB']].corr(), annot=True, cmap='coolwarm')

# Scatter plots
sns.scatterplot(data=df, x='WS', y='GHI')
sns.scatterplot(data=df, x='RH', y='Tamb')


## Wind & Distribution Analysis

In [ ]:
# Histogram example
df['GHI'].hist(bins=50)
plt.show()

# Wind rose (requires windrose library)
# pip install windrose
from windrose import WindroseAxes
ax = WindroseAxes.from_ax()
ax.bar(df['WD'], df['WS'], normed=True, opening=0.8, edgecolor='white')
ax.set_legend()
plt.show()


## Temperature & Bubble Chart

In [ ]:
plt.scatter(df['Tamb'], df['GHI'], s=df['RH']*2, alpha=0.5)
plt.xlabel('Tamb')
plt.ylabel('GHI')
plt.title('GHI vs Tamb (bubble=RH)')
plt.show()
